###  Derin Öğrenme İle Toprak Sınıflandırma

Zeminlerin sınıflandırılması, geoteknik mühendisliği, ziraat bilimi ve inşaat dahil olmak üzere birçok alanda önemli bir yetenek olabilir.
Birçok sınıflandırma şeması mevcut olmakla birlikte, toprakları sınıflandırmanın popüler ve sezgisel bir yolu, çakıl, silt, kum vb. gibi toprak tane boyutuna dayanmaktadır. Bu, toprağı sınıflandırmanın oldukça basit bir yoludur çünkü her sınıflandırma arasında kesin sınırlar vardır. Örneğin, çakıl çapı 2 mm'den büyük herhangi bir tanedir, kum  $\frac{1}{16}$ mm ile 2 mm arasında, ve silt/kil  $\frac{1}{16}$ mm'den daha küçüktür.

<img align=left width=400px src='https://apmonitor.com/pds/uploads/Main/soil_classification.png'>

### Amaç
Bu alıştırmada bilgisayar görüşü, çakıl, kum ve silt fotoğraflarına dayalı olarak zeminlerin bir sınıflandırma modelini yapar. Toprağın fotoğraflarını sınıflandırmak için evrişimli bir sinir ağı çerçevesi kullanılmıştır.

### Anahtar kavramlar:
#### Veri Büyütme
Veri Büyütme, mevcut verilerden yeni veri kümeleri oluşturarak mevcut görüntü veri kümesini büyüten, görüntü sınıflandırmada yaygın olarak kullanılan bir uygulamadır. Bu büyütme en çok görüntüyü kaydırma, görüntüyü yatay veya dikey olarak yansıtma, görüntüyü yakınlaştırma vb. işlemlerde görülür. Bu büyütmeler daha sonra yeni görüntüler oluşturur ve böylece üzerinde çalışılacak görüntü sayısını artırır. Bu konuda daha fazla bilgi edinmek için okuyun [article from Jason Brownlee](https://machinelearningmastery.com/how-to-configure-image-data-augmentation-when-training-deep-learning-neural-networks/).

#### Evrişimsel Sinir Ağı (CNN)
Bir evrişimli sinir ağı (CNN), görüntü sınıflandırması için popüler bir modeldir. CNN'ler, bir görüntüdeki piksel değerleri arasında dolaşarak ve bir filtre "çekirdek" matrisi ile bu piksellerin nokta çarpımını hesaplayarak bir görüntünün özelliklerini ayırt eder. Bu çekirdek matrisleri, görüntünün dikey ve yatay çizgiler, eğrilik vb. gibi farklı yönlerini vurgular.

In [ ]:
training_data_directory = 'train'
test_data_directory = 'test'

### Kurulum

Aşağıdaki Python modüllerini içe aktarın. Eksik paketleri kurmak için ```pip``` kullanmayı unutmayın. 

In [ ]:
import os
import re
import cv2
import time
import shutil
import zipfile
import urllib.request
import numpy as np
from PIL import Image
from os import listdir
from os.path import isfile, join
from random import randrange
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D

### Veri

Veriler, test ve eğitim dizinlerine ayrılır. Test klasörü ve tren klasörü, olası kirlere karşılık gelen alt dizinleri içerir. Klasörlerin etiketli ağaç yapısı, eğitim ve test için fotoğrafların işaretlenmesine yardımcı olur.

In [ ]:
# soil_photos.zip Veri İndirme
file = 'soil_photos.zip'
url = 'http://apmonitor.com/pds/uploads/Main/'+file
urllib.request.urlretrieve(url, file)

# soil_photos.zip Arşivini dışarı aktarıp zip dosyasının silinmesi
with zipfile.ZipFile(file, 'r') as zip_ref:
    zip_ref.extractall('./')
os.remove(file)

Verileri Python oturumuna aktarın. İlk adım, görüntüleri bir formata dönüştürmektir 1) verileri model için okunabilir hale getirir, ve 2) modelin öğrenmesi için daha fazla eğitim materyali sağlar. Örneğin, "training_data_processor" değişkeni, verileri bir model girdisi olacak şekilde ölçeklendirir, ancak aynı zamanda her bir görüntüyü alır ve modelin aynı resmin birden çok varyasyonundan öğrenebilmesi için onu büyütür. Modelin yön veya boyuttan ziyade toprak fotoğrafından öğrenmesi için yatay olarak çevirir, döndürür, kaydırır ve daha fazlasını yapar.

In [ ]:
# Veri işleme araçlarını tanımlıyoruz
training_data_processor = ImageDataGenerator(
    rescale = 1./255,
    horizontal_flip = True,
    zoom_range = 0.2,
    rotation_range = 10,
    shear_range = 0.2,
    height_shift_range = 0.1,
    width_shift_range = 0.1
)

test_data_processor = ImageDataGenerator(rescale = 1./255)

# Verileri içeri alıyoruz 
training_data = training_data_processor.flow_from_directory(
    training_data_directory,
    target_size = (256, 256),
    batch_size = 32,
    class_mode = 'categorical',
)

testing_data = test_data_processor.flow_from_directory(
    test_data_directory,
    target_size = (256 ,256),
    batch_size = 32,
    class_mode = 'categorical',
    shuffle = False
)

### Model Oluşturma

CNN modelini oluşturun. Aşağıdaki hücre, model oluşturma için parametreleri ayarlar. Bu, evrişimli katmanların sayısını, tam bağlantılı yoğun katmanları, her katmandaki düğüm sayısını ve eğitim dönemlerinin sayısını içerir. "layer_size" parametresinin modelin eğitim hızı, doğruluğu ve boyutu üzerinde büyük etkisi vardır.

In [ ]:
# model parametrelerini seç
num_conv_layers = 2
num_dense_layers = 1
layer_size = 32
num_training_epochs = 20
MODEL_NAME = 'soil'

In [ ]:
# Model değişkenini başlat
model = Sequential()

# model değişkenine özellikler ekliyoruz
model.add(Conv2D(layer_size, (3, 3), input_shape=(256,256, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

# num_conv_layers'a dayalı ek evrişim katmanları ekleyoruz
for _ in range(num_conv_layers-1):
    model.add(Conv2D(layer_size, (3, 3)))
    model.add(Activation('relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))

# boyutsallığı azaltıyoruz
model.add(Flatten())

# belirtilmişse tamamen bağlı "yoğun" katmanlar ekliyoruz
for _ in range(num_dense_layers):
    model.add(Dense(layer_size))
    model.add(Activation('relu'))

# çıkış layerını ayarlıyoruz
model.add(Dense(3))
model.add(Activation('softmax'))

# tüm eklenen özelliklerle sıralı modeli derliyoruz
model.compile(loss='categorical_crossentropy',
                optimizer='adam',
                metrics=['accuracy'],
                )

# modeli eğitmek/ayarlamak için yüklenmiş verileri kullanıyoruz
model.fit(training_data,
            epochs=num_training_epochs,
            validation_data = testing_data)

# eğitilmiş modeli kaydediyoruz
model.save(f'{MODEL_NAME}.h5')

### Model Test Aşaması

Model artık eğitilmiş ve bilgisayara kaydedilmiştir (bu not defteri ile aynı klasörde). Yukarıdaki yazdırılan çıktının son satırı, hem eğitim verilerinin hem de doğrulama veya test verilerinin doğruluğunu içerir. Modelin eğitilmediği görüntülerdeki doğruluğa karşılık gelen son satırdaki ``val_accuracy`` değerine dikkat edin. Bu, diğer görüntülerdeki tahmin edici performansının en iyi ölçüsüdür.

Aşağıdaki işlev olan ``make_prediction``, girdi olarak bir zemin fotoğrafına giden dosya yolunu alır ve modelin öngördüğü sınıflandırmayı verir. Test klasöründen, dosya yolunu ikinci hücredeki test_image_filepath değişkenine kopyalayın. Birkaç fotoğraf deneyin ve modelin nasıl oluştuğunu görün. 

In [ ]:
def make_prediction(image_fp):
    im = cv2.imread(image_fp) # resim yüklüyoruz
    plt.imshow(im[:,:,[2,1,0]])
    img = image.load_img(image_fp, target_size = (256,256))
    img = image.img_to_array(img)

    image_array = img / 255. # resmi ölçeklendiriyoruz
    img_batch = np.expand_dims(image_array, axis = 0)
    
    class_ = ["Gravel", "Sand", "Silt"] # çıkış değerleri tanımlıyoruz
    predicted_value = class_[model.predict(img_batch).argmax()]
    true_value = re.search(r'(Gravel)|(Sand)|(Silt)', image_fp)[0]
    
    out = f"""Predicted Soil Type: {predicted_value}
    True Soil Type: {true_value}
    Correct?: {predicted_value == true_value}"""
    
    return out

In [ ]:
test_image_filepath = test_data_directory + r'/Sand/0.jpg'
print(make_prediction(test_image_filepath))

### Zemin Sınıflandırma Yüzdeleri

Topraklar tam olarak homojen değildir (hepsi bir toprak tipi). Topraklar genellikle türlerin bir karışımıdır ve yüzde kullanılarak daha iyi temsil edilebilir. Örneğin, aşağıdaki hücrede "Silt" etiketli bir test fotoğrafı gösterilmektedir:

In [ ]:
percentage_photo = test_data_directory + r"/Silt/5.jpg"
im = cv2.imread(percentage_photo) # Test etmek için fotoğrafı içeri aktarıyoruz
plt.imshow(im[:,:,[2,1,0]])

Bu fotoğrafta biraz çakıl var, ancak tek bir fotoğrafta kum, çakıl ve silt var gibi görünüyor. Bu görüntüyü daha iyi sınıflandırmak için her etiketin bir oranını oluşturun. Örneğin, bu fotoğraf %30 Çakıl, %20 Kum ve %50 Silt olabilir.

Fotoğrafı birçok küçük parçaya veya kareye bölün ve daha küçük kareler üzerinde bir sınıflandırıcı eğitin. Her küçük kareyi sınıflandırmak için fotoğrafta gezinin ve Çakıl, Kum ve Silt ile ilgili karelerin bir oranını alın. Bu oran daha sonra ilgili toprak tipinin bir yüzdesine dönüştürülür.

### Veri

Mevcut eğitim fotoğraflarını daha küçük bölümlere ayırmak için aşağıdaki hücreleri çalıştırın. Yeni dizin, train_divided ve test_divided olarak adlandırılacaktır.  

Not: Bu biraz zaman alacaktır (~30 saniye ile 2 dakika arası)

In [ ]:
def split_images(image_dir, save_dir):
    classification_list = ['Gravel', 'Sand', 'Silt']
    for classification in classification_list:
        folder = image_dir + '/' + classification + '/'
        save_folder = save_dir + '/' + classification + '/'
        files = [f for f in listdir(folder) if isfile(join(folder, f))]

        for file in files:
            if '.ini' in file:
                continue
            fp = folder + file
            img = cv2.imread(fp)
            h,w,c = img.shape
            im_dim = 64
            # Görüntüleri kırpıyoruz
            for r in range(0,img.shape[0],im_dim):
                for c in range(0,img.shape[1],im_dim):
                    cropped_img = img[r:r+im_dim, c:c+im_dim,:]
                    ch, cw, cc = cropped_img.shape
                    if ch == im_dim and cw == im_dim:
                        write_path = f"{save_folder + str(randrange(100000))}img{r}_{c}.jpg"
                        cv2.imwrite(write_path,cropped_img)
                    else:
                        pass

In [ ]:
try:
    parent = training_data_directory.replace('train', '')
    dirs = ['train_divided', 'test_divided']
    class_ = ["Gravel", "Sand", "Silt"]
    for dir in dirs:
        os.mkdir(os.path.join(parent, dir))
        for classification in class_:
            os.mkdir(os.path.join(parent, dir, classification))

    # kırpılmış eğitim görüntüleri
    split_images(image_dir=training_data_directory,
                save_dir=training_data_directory.replace('train', 'train_divided'))
    # kırpılmış test görüntüleri
    split_images(image_dir=test_data_directory,
                save_dir=test_data_directory.replace('test', 'test_divided'))
except FileExistsError:
    pass

### Model Yükleme

Giriş piksellerinin sayısı değişirse, yeni bir model eğitilmelidir veya görüntü orijinal eğitim boyutlarına yeniden ölçeklendirilmelidir. Bu demonun amaçları doğrultusunda, her görüntüde 256x256 blok (16) alt örneklemeden önce fotoğraflar 1024x1024'e yükseltildiğinden yeniden eğitim gerekli değildir.

In [ ]:
model_fp = os.getcwd()+'/'+'soil.h5'
print(model_fp)
model = load_model(model_fp)

### Görüntü Sınıflandırma

Bir test görüntüsünü sınıflandırmak için yüklenen modeli kullanın. 'classify_images' işlevi bir görüntü ve bir model alır ve her 256x256 karede döngü yapar. Her kareyi sınıflandırır ve kesirli toprak tahmini oluşturmak için sayaca ekler. İşlev, sınıflandırılan her toprak tipinin oranını verir.

In [ ]:
def classify_images(image_fp, model):
    classes = ['Gravel', 'Sand', 'Silt']
    gravel_count = 0
    sand_count = 0
    silt_count = 0

    img = cv2.imread(image_fp)
    img = cv2.resize(img,(1024,1024))
    im_dim = 256

    for r in range(0, img.shape[0], im_dim):
        for c in range(0, img.shape[1], im_dim):
            cropped_img = img[r:r + im_dim, c:c + im_dim, :]
            h, w, c = cropped_img.shape
            if h == im_dim and w == im_dim:
                classification = model_classify(cropped_img, model)
                if classification == classes[0]:
                    gravel_count += 1
                elif classification == classes[1]:
                    sand_count += 1
                elif classification == classes[2]:
                    silt_count += 1
            else:
                continue
    total_count = gravel_count + sand_count + silt_count
    proportion_array = [gravel_count / total_count, sand_count / total_count, silt_count / total_count]
    return proportion_array


def model_classify(cropped_img, model):
    classes = ['Gravel', 'Sand', 'Silt']
    image_array = cropped_img / 255.
    img_batch = np.expand_dims(image_array, axis=0)
    prediction_array = model.predict(img_batch)[0]
    first_idx = np.argmax(prediction_array)
    first_class = classes[first_idx]
    return first_class

def classify_percentage(image_fp):
    start = time.time()
    out = classify_images(image_fp=image_fp, model=model)
    finish = str(round(time.time() - start, 5))
    
    im = cv2.imread(image_fp) # load image
    plt.imshow(im[:,:,[2, 1, 0]])

    print(f'''---
Çakıl Yüzdesi: {round(out[0] * 100, 2)}%)
Kum Yüzdesi: {round(out[1] * 100, 2)}%)
Silt Yüzdesi: {round(out[2] * 100, 2)}%)
Sınıflandırma Zamanı: {finish} seconds
---''')

In [ ]:
classify_percentage(image_fp=percentage_photo)

Yüzde sınıflandırılmış fotoğrafla eşleşiyor mu? Toprak sınıflandırmasına yönelik her iki yaklaşımın da avantajları vardır. Örneğin, tüm görüntüyü sınıflandırmak daha hızlıdır. Bununla birlikte, yüzde sınıflandırması hesaplama açısından daha pahalıdır ancak daha fazla bilgi sağlar ve gerçek sınıflandırmanın daha iyi bir temsili olabilir.